In [2]:
import torch
import glob
import tqdm

import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import torch.nn.functional as F

from utilities import utils, train_utils
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize, MinMaxScaler
from torch.utils.data import DataLoader, TensorDataset, Dataset

In [3]:
# for testing: copy a (1, 768) vector 77 times to create (77, 768)
class CustomDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        sample = self.data[idx][0].repeat(77, 1)
        sample = sample.unsqueeze(0)

        return sample

In [4]:
paths = sorted(glob.glob('./data/000*/*_embedding.msgpack'))
device = train_utils.get_device()

X = []
for path in tqdm.tqdm(paths):
    pos_emb, neg_emb = utils.read_embedding_data(path)
    X.append(pos_emb)
X = np.concatenate(X, axis=0)

scl = MinMaxScaler()

# reshape to 2D to apply scaling
# X = X.reshape(-1, 768)
# X, l2_norm = normalize(X, norm='l2', axis=1, return_norm=True)

Xtr, Xvl = train_test_split(X, test_size=0.2, random_state=42)

Xtr = Xtr.reshape(-1, 768)
Xvl = Xvl.reshape(-1, 768)

Xtr = scl.fit_transform(Xtr)
Xvl = scl.transform(Xvl)

# reshape it back
Xtr = Xtr.reshape(-1, 77, 768)
Xvl = Xvl.reshape(-1, 77, 768)

train_data = torch.tensor(Xtr, dtype=torch.float32)
val_data = torch.tensor(Xvl, dtype=torch.float32)

train_dataset = TensorDataset(train_data)
val_dataset = TensorDataset(val_data)
# train_dataset = CustomDataset(train_data)
# val_dataset = CustomDataset(val_data)

  0%|          | 0/8964 [00:00<?, ?it/s]

100%|██████████| 8964/8964 [00:49<00:00, 181.48it/s]


In [7]:
class Encoder(nn.Module):
    def __init__(self, input_dim, latent_dim, pooling='max'):
        super(Encoder, self).__init__()
        self.input_dim = input_dim
        self.latent_dim = latent_dim

        self.layer_norm = nn.LayerNorm(self.input_dim)
        
        # define encoder layers: 3 layers fc of 1024, last fc no activation
        self.encoder_layers = nn.Sequential(
            nn.Linear(self.input_dim, 2048),
            nn.ReLU(),
            nn.Linear(2048, 2048),
            nn.ReLU(),
            nn.Linear(2048, 1024),
            nn.ReLU(),
            nn.Linear(1024, 1024),
            nn.ReLU(),
        )

        self.fc_latent = nn.Linear(1024, self.latent_dim)
        if pooling == 'max':
            self.pooling = nn.MaxPool1d(kernel_size=77, stride=1)
        elif pooling == 'avg':
            self.pooling = nn.AvgPool1d(kernel_size=77, stride=1)

    def forward(self, x):
        assert x.shape == (x.shape[0], 77, 768)

        # x = self.layer_norm(x)

        # reshape for fc
        x = x.view(-1, 768)
        x = self.encoder_layers(x)

        # reshape back to batch_size * 77 x hidden size
        x = x.view(-1, 77, 1024)
        x = self.pooling(x.permute(0, 2, 1)).squeeze(-1)
        x = self.fc_latent(x)

        return x
    
class Decoder(nn.Module):
    def __init__(self, input_dim, latent_dim, out_act='tanh'):
        super(Decoder, self).__init__()
        self.input_dim = input_dim
        self.latent_dim = latent_dim

        # define decoder layers: 3 layers of fc 1024, last layer activated with tanh
        self.decoder_layers = nn.Sequential(
            nn.Linear(self.latent_dim, 1024),
            nn.ReLU(),
            nn.Linear(1024, 1024),
            nn.ReLU(),
            nn.Linear(1024, 2048),
            nn.ReLU(),
            nn.Linear(2048, 2048),
            nn.ReLU(),
            nn.Linear(2048, 77 * self.input_dim),
        )

        if out_act == 'tanh':
            self.out_act = nn.Tanh()
        elif out_act == 'sigmoid':
            self.out_act = nn.Sigmoid()
        elif out_act == None:
            self.out_act = None

    def forward(self, z):
        assert z.shape == (z.shape[0], self.latent_dim)

        x = self.decoder_layers(z)
        if self.out_act:
            x = self.out_act(x)

        # reshape to batch_size * 77 * 768
        x = x.view(-1, 77, 768)

        return x
    
class AutoEncoder(nn.Module):
    def __init__(self, input_dim, latent_dim, pooling='max', out_act='tanh'):
        super(AutoEncoder, self).__init__()

        self.encoder = Encoder(input_dim, latent_dim, pooling)
        self.decoder = Decoder(input_dim, latent_dim, out_act)

    def forward(self, x):
        self.z = self.encoder(x)
        recon = self.decoder(self.z)

        return recon

In [12]:
model = AutoEncoder(input_dim=768, latent_dim=768, pooling='avg', out_act='sigmoid')
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=10, threshold=0.01)
model = model.to(device)
model

AutoEncoder(
  (encoder): Encoder(
    (layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (encoder_layers): Sequential(
      (0): Linear(in_features=768, out_features=2048, bias=True)
      (1): ReLU()
      (2): Linear(in_features=2048, out_features=2048, bias=True)
      (3): ReLU()
      (4): Linear(in_features=2048, out_features=1024, bias=True)
      (5): ReLU()
      (6): Linear(in_features=1024, out_features=1024, bias=True)
      (7): ReLU()
    )
    (fc_latent): Linear(in_features=1024, out_features=768, bias=True)
    (pooling): AvgPool1d(kernel_size=(77,), stride=(1,), padding=(0,))
  )
  (decoder): Decoder(
    (decoder_layers): Sequential(
      (0): Linear(in_features=768, out_features=1024, bias=True)
      (1): ReLU()
      (2): Linear(in_features=1024, out_features=1024, bias=True)
      (3): ReLU()
      (4): Linear(in_features=1024, out_features=2048, bias=True)
      (5): ReLU()
      (6): Linear(in_features=2048, out_features=2048, bias=True)

In [13]:
save_path = 'experiments/fc_exp7/'
epoch = 1500
batch_size = 128
sparsity_penalty_weight = 0

In [14]:
# loss function
class MSELossFC(nn.Module):
    def __init__(self, sparsity_penalty_weight=0.0):
        super(MSELossFC, self).__init__()
        self.sparsity_penalty_weight = sparsity_penalty_weight

    def forward(self, y, y_hat, z):
        assert y.shape == (y.shape[0], 77, 768)
        assert y_hat.shape == (y.shape[0], 77, 768)

        reconstruction_loss = nn.MSELoss(reduction='sum')(y, y_hat)
        
        l1_penalty = torch.abs(z).mean()
        sparsity_loss = l1_penalty * self.sparsity_penalty_weight

        total_loss = reconstruction_loss + sparsity_loss

        return total_loss

criterion = MSELossFC(sparsity_penalty_weight)

In [15]:
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

train_utils.train_loop(
    train_loader=train_loader,
    val_loader=val_loader,
    model=model,
    criterion=criterion,
    device=device,
    optimizer=optimizer,
    epochs=epoch,
    save_path=save_path,
    scheduler=scheduler
)

val loss: 711.273: 100%|██████████| 15/15 [00:00<00:00, 23.34it/s]


Weight saved: epoch 0
Epoch 0	Train Loss: 711.795  Val Loss: 685.658



val loss: 701.289: 100%|██████████| 15/15 [00:00<00:00, 26.21it/s]


Weight saved: epoch 1
Epoch 1	Train Loss: 681.346  Val Loss: 681.798



val loss: 700.449: 100%|██████████| 15/15 [00:00<00:00, 23.01it/s]


...

Weight saved: epoch 952
Epoch 952	Train Loss: 605.395  Val Loss: 611.253



val loss: 676.316: 100%|██████████| 15/15 [00:00<00:00, 26.61it/s]


Weight saved: epoch 953
Epoch 953	Train Loss: 605.243  Val Loss: 611.253



train loss: 609.127:  32%|███▏      | 18/57 [00:02<00:06,  6.32it/s]


KeyboardInterrupt: 

In [17]:
model.eval()
with torch.no_grad():
    recon = model(val_data.to(device))

criterion(val_data.to(device), recon, model.z) / 800 / 77

tensor(17.6577, device='cuda:0')